# osm2gmns  
osm2gmns is a Python package that facilitates downloading networks from [OpenStreetMap](https://www.openstreetmap.org) (OSM), and save them in .csv files in [GMNS](https://github.com/zephyr-data-specs/GMNS) format.  

This is quick guide to getting started with osm2gmns. For more details, check osm2gmns's [official documentation](https://osm2gmns.readthedocs.io/en/latest/index.html)

### 1. Install osm2gmns
You can run this command from the command line, or just run the code cell below

In [ ]:
pip install osm2gmns

### 2. Import the module

In [ ]:
import osm2gmns as og

In [ ]:
# OSM network data
target = 'ucf.osm'
id = 1159228

### 3. Download the Network  

`downloadOSMData()` requests the network data from OSM through the Overpass API and saves it to a file.  

TODO: add note about using osmosis for smaller downloads

In [ ]:
og.downloadOSMData(area_id=id, output_filename=target)

Note: The .osm file obtained from the code above might be too large to work with for particularly big networks. One way you can reduce the network size is to use `osmosis`, a command line Java application used for processing OSM data.

The Overpass API used by `downloadOSMData()` tries to reduce the size of the data it sends by omitting unnecessary attributes added to the OSM 0.6 format, such as node.version. However, osmosis requires that all these attributes be present before it can process the data. 

We can add dummy data to the OSM file with the `osmconvert` command line application, with the following command:
`$ osmconvert "source.osm" --fake-author -o="dest.osm"`

After adding the dummy data, we can filter the network file to include highways only using the following command:  
`$ osmosis --read-xml "dest.osm" --tf accept-ways highway=* --used-node --write-xml "highways.osm"`

In [ ]:
# define separate target file after preparing highways data
highways_target = 'highways.osm'

### 4. Load the Network  
`getNetFromFile()` loads a osm file `filename`, and returns a Network object.  
For a simple visualization of the network, you can use the function `show()`

TODO: describe `getNetFromFile()` arguments

In [ ]:
net = og.getNetFromFile(filename=target, POI=True)
og.show(net, figsize=(4,4))

In [ ]:
# highways only data may be more efficient to work with for particularly large networks
highways = og.getNetFromFile(highways_target)
# og.show(highways, figsize=(4,4))

use `connectPOIWithNet()` to connect the network with the Points of Interest (POI) avaliable

In [ ]:
og.connectPOIWithNet(net)
# og.show(net, figsize=(4,4))

### 5. Save GMNS Network
Use `outputNetToCSV()` to save the network in GMNS format.  
The network files will be saved to `output_folder`

In [ ]:
csv_dir = 'gmns'
og.outputNetToCSV(net, output_folder=csv_dir)

### 6. Consolidate Intersections  
Some intersections in the network may be represented by more than one node.  
To simplify the network, you can use `consolidateComplexIntersections()`.  

setting the `auto_identify` argument to `True` will make osm2gmns automatically consolidate any complex intersection.  
Alternatively, you can pass a path to an intersection file to the argument `intersection_path` specifying the central position of each complex intersection.  

In [ ]:
og.consolidateComplexIntersections(net, auto_identify=True)
og.outputNetToCSV(net, output_folder=csv_dir+"_consolidated")
og.show(net, figsize=(4,4))

### 7. Multi Resolution Networks  
You can build the network at different resolutions using `buildMultiResolutionNets()`, down to mesoscopic and microscopic networks.

In [ ]:
og.buildMultiResolutionNets(net)
og.outputNetToCSV(net, output_folder=csv_dir+"_resolutions")

### 8. Other Useful Functions

`generateMovements(network)` uses built-in methods to generate movements for each node in the network.  

Note: this function requires that the links have lanes information. You can set `default_lanes` to `True` when loading the network from disk if your dataset doesn't have lane information.

In [ ]:
net = og.getNetFromFile(target, default_lanes=True, POI=True)
og.consolidateComplexIntersections(net)

og.generateMovements(net)
og.outputNetToCSV(net, output_folder=csv_dir+"_movements")

`loadNetFromCSV()` can be used to load a network if you already have it saved in GMNS format.

In [ ]:
net = og.loadNetFromCSV(
    folder=csv_dir+"_movements",
    node_file='node.csv',
    link_file='link.csv',
    movement_file='movement.csv',
    POI_file='poi.csv'
    )

og.show(net, figsize=(4,4))

`saveFig()` saves the network plot with the specified filepath

In [ ]:
og.saveFig(net, picpath=csv_dir+"_movements/network.jpg")

Use `generateNodeActivityInfo()` to generate activity information such as activity_type, is_boundary, zone_id for nodes.

In [ ]:
og.generateNodeActivityInfo(net)
og.outputNetToCSV(net, output_folder=csv_dir+"_activity")

`generateLinkVDFInfo(network)` generates VDF information, such as `VDF_fftt1` (free flow travel time of link in minutes) and `VDF_cap1` (link capacity in vehicles/hour/link)

In [ ]:
og.generateLinkVDFInfo(net)
og.outputNetToCSV(net, output_folder=csv_dir+"_vdf")

`combineShortLinks(network)` combines links connected by two-degree nodes (explanation) into a longer link

In [ ]:
og.combineShortLinks(net)
og.outputNetToCSV(net, output_folder=csv_dir+"_combined")